# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    beta = 0
    total_invested = P["Amount Invested"].sum()
    for i in range(0, len(P)):
        beta += P.iloc[i, 0] * (P.iloc[i, 2] / total_invested)
    return rf + beta * mkt_rp

#expected_return(portfolio, 0.02, 0.05)

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    # Write your code here
    # Extract data from the portfolio DataFrame
    betas = P["Beta"].values
    volatilities = P["Volatility"].values
    amounts_invested = P["Amount Invested"].values

    # Calculate weights
    weights = amounts_invested / np.sum(amounts_invested)

    # Calculate the covariance matrix
    cov_matrix = np.diag(volatilities ** len(P))

    # Iterate over all pairs of assets to update off-diagonal elements with correlation
    num_assets = len(betas)
    for i in range(num_assets):
        for j in range(i + 1, num_assets):
            cov_matrix[i, j] = cor * volatilities[i] * volatilities[j]
            cov_matrix[j, i] = cov_matrix[i, j]

    # Calculate portfolio variance
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))

    # Calculate portfolio volatility (standard deviation)
    portfolio_volatility = np.sqrt(portfolio_variance)

    return portfolio_volatility

#volatility(portfolio, -0.2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    expected_return = rf + (mkt_rp / mkt_vol) * vol
    return expected_return

#expected_return2(0.02, 0.05, 0.16, 0.2408)

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    returns = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)
    factors["Mkt-RF"] = factors["Mkt-RF"] / 100
    factors["RF"] = factors["RF"] / 100
    returns["month2"] = returns["month"].map("{:02}".format)
    returns["date"] = returns["year"].astype("string") + returns["month2"]
    #display(returns)
    #display(factors)
    groups = returns.groupby("CUSIP")
    betas = []
    for g, data in groups:
        covariances = []
        rfs = []
        rmkts = []
        for i in range(0, len(data)):
            rf = factors["RF"].loc[factors["date"] == data.iloc[i, 5]]
            rfs.append(rf)
            rmkt = factors["Mkt-RF"].loc[factors["date"] == data.iloc[i, 5]]
            rmkts.append(rmkt)
            covariances.append(np.cov(data.iloc[i, 1] - rf))
        data["covariance"] = pd.Series(covariances)
        data["rf"] = pd.Series(rfs)
        data["rmkt"] = pd.Series(rmkts)
        data["difference"] = data.rmkt - data.rf
        data["variance"] = data["difference"].var()
        data["beta"] = data["covariance"] / data["variance"]
        betas.append(data["beta"].mean())
    
    beta = pd.Series(betas)
    output = pd.DataFrame(data = {
        "CUSIP" : returns.groupby("CUSIP")["CUSIP"].nth(0),
        "EXP_RETURN" : returns.groupby("CUSIP")["RET"].mean(),
        "BETA" : beta
    })
    # Cov(R_i - Rf, R_mkt – Rf) / Var(R_mkt - Rf)
    return output

#CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    
    return data["EXP_RETURN"].mean()